In [1]:
import melex

In [3]:
dataset = melex.Dataset("dev_data/dataset_test")
assert dataset.is_valid()
print(f"Length of dataset: {len(dataset)}")

Length of dataset: 186


In [ ]:
melodies, eval_df = melex.inference_pipeline(
    dataset,
    save_plot=True,
    save_midi=True,
    save_excel=True)

No save directory provided, using default: outputs\20250730_184026


Processing songs (n_jobs=12):   0%|          | 0/186 [00:00<?, ?it/s]

Song <吻得太逼真(张敬轩)> generated an exception: bad allocation
Song <星辰大海(黄霄雲)> generated an exception: bad allocation
Song <星空(五月天)> generated an exception: 
Song <有点甜(汪苏泷、BY2)> generated an exception: 
Song <明天会更好(华语群星)> generated an exception: 
Song <朋友身份(胡鸿钧)> generated an exception: 
Song <晴天(周杰伦)> generated an exception: 
Song <毒药(萧秉治)> generated an exception: 
Song <有生一天(周柏豪)> generated an exception: 
Song <最真心一对(何雁诗)> generated an exception: 
